In [1]:
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.geometry import Polygon

# Step 1: 读取tfw文件
def read_tfw(tfw_path):
    with open(tfw_path, 'r') as f:
        lines = f.readlines()
        x_pixel_size = float(lines[0])
        rotation_1 = float(lines[1])
        rotation_2 = float(lines[2])
        y_pixel_size = float(lines[3])
        top_left_x = float(lines[4])
        top_left_y = float(lines[5])
    return top_left_x, top_left_y, x_pixel_size, y_pixel_size

# Step 2: 读取图像尺寸
def read_image_size(tif_path):
    dataset = gdal.Open(tif_path)
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    return width, height

# Step 3: 读取YOLO标签文件
def read_yolo_labels(label_path):
    labels = []
    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            cls = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            labels.append((cls, x_center, y_center, width, height))
    return labels

# Step 4: 将像素坐标转换为地理坐标
def pixel_to_geo(x_pix, y_pix, top_left_x, top_left_y, x_pixel_size, y_pixel_size):
    x_geo = top_left_x + x_pix * x_pixel_size
    y_geo = top_left_y + y_pix * y_pixel_size
    return x_geo, y_geo

# Step 5: 创建Shapefile
def create_shapefile(output_path, labels, top_left_x, top_left_y, x_pixel_size, y_pixel_size, img_width, img_height):
    driver = ogr.GetDriverByName("ESRI Shapefile")
    data_source = driver.CreateDataSource(output_path)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)  # 根据你的坐标系修改
    layer = data_source.CreateLayer("labels", srs, ogr.wkbPolygon)
    
    # 创建属性表
    field_class = ogr.FieldDefn("Class", ogr.OFTInteger)
    layer.CreateField(field_class)
    
    for label in labels:
        cls, x_center, y_center, width, height = label
        
        # 计算实际像素坐标
        x_pix_center = x_center * img_width
        y_pix_center = y_center * img_height
        box_width = width * img_width
        box_height = height * img_height
        
        # 计算四个角的地理坐标
        x_pix_min = x_pix_center - box_width / 2
        y_pix_min = y_pix_center - box_height / 2
        x_pix_max = x_pix_center + box_width / 2
        y_pix_max = y_pix_center + box_height / 2
        
        geo_coords = [
            pixel_to_geo(x_pix_min, y_pix_min, top_left_x, top_left_y, x_pixel_size, y_pixel_size),
            pixel_to_geo(x_pix_max, y_pix_min, top_left_x, top_left_y, x_pixel_size, y_pixel_size),
            pixel_to_geo(x_pix_max, y_pix_max, top_left_x, top_left_y, x_pixel_size, y_pixel_size),
            pixel_to_geo(x_pix_min, y_pix_max, top_left_x, top_left_y, x_pixel_size, y_pixel_size),
            pixel_to_geo(x_pix_min, y_pix_min, top_left_x, top_left_y, x_pixel_size, y_pixel_size)
        ]
        
        # 创建多边形
        poly = Polygon(geo_coords)
        wkt = poly.wkt
        feature = ogr.Feature(layer.GetLayerDefn())
        feature.SetField("Class", cls)
        geom = ogr.CreateGeometryFromWkt(wkt)
        feature.SetGeometry(geom)
        layer.CreateFeature(feature)
        feature = None
    
    data_source = None

# 使用示例
tif_path = "tif_path.tif"
tfw_path = "tfw_path.tfw"
label_path = "yolo.txt"
output_shp_path = "output_dir_path.shp"

top_left_x, top_left_y, x_pixel_size, y_pixel_size = read_tfw(tfw_path)
img_width, img_height = read_image_size(tif_path)
labels = read_yolo_labels(label_path)
create_shapefile(output_shp_path, labels, top_left_x, top_left_y, x_pixel_size, y_pixel_size, img_width, img_height)

In [1]:
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.geometry import Polygon

# 读取tfw文件
def read_tfw(tfw_path):
    with open(tfw_path, 'r') as f:
        lines = f.readlines()
        x_pixel_size = float(lines[0])
        rotation_1 = float(lines[1])
        rotation_2 = float(lines[2])
        y_pixel_size = float(lines[3])
        top_left_x = float(lines[4])
        top_left_y = float(lines[5])
    return top_left_x, top_left_y, x_pixel_size, y_pixel_size

# 读取图像尺寸
def read_image_size(tif_path):
    dataset = gdal.Open(tif_path)
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    return width, height

# 读取YOLO标签文件
def read_yolo_labels(label_path):
    labels = []
    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            cls = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            labels.append((cls, x_center, y_center, width, height))
    return labels

# 将像素坐标转换为地理坐标
def pixel_to_geo(x_pix, y_pix, top_left_x, top_left_y, x_pixel_size, y_pixel_size):
    x_geo = top_left_x + x_pix * x_pixel_size
    y_geo = top_left_y + y_pix * y_pixel_size
    return x_geo, y_geo

# 创建Shapefile
def create_shapefile(output_path, labels, top_left_x, top_left_y, x_pixel_size, y_pixel_size, img_width, img_height):
    driver = ogr.GetDriverByName("ESRI Shapefile")
    data_source = driver.CreateDataSource(output_path)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)  # 根据你的坐标系修改
    layer = data_source.CreateLayer("labels", srs, ogr.wkbPolygon)
    
    # 创建属性表
    field_class = ogr.FieldDefn("Class", ogr.OFTInteger)
    layer.CreateField(field_class)
    
    for label in labels:
        cls, x_center, y_center, width, height = label
        
        # 计算实际像素坐标
        x_pix_center = x_center * img_width
        y_pix_center = y_center * img_height
        box_width = width * img_width
        box_height = height * img_height
        
        # 计算四个角的地理坐标
        x_pix_min = x_pix_center - box_width / 2
        y_pix_min = y_pix_center - box_height / 2
        x_pix_max = x_pix_center + box_width / 2
        y_pix_max = y_pix_center + box_height / 2
        
        geo_coords = [
            pixel_to_geo(x_pix_min, y_pix_min, top_left_x, top_left_y, x_pixel_size, y_pixel_size),
            pixel_to_geo(x_pix_max, y_pix_min, top_left_x, top_left_y, x_pixel_size, y_pixel_size),
            pixel_to_geo(x_pix_max, y_pix_max, top_left_x, top_left_y, x_pixel_size, y_pixel_size),
            pixel_to_geo(x_pix_min, y_pix_max, top_left_x, top_left_y, x_pixel_size, y_pixel_size),
            pixel_to_geo(x_pix_min, y_pix_min, top_left_x, top_left_y, x_pixel_size, y_pixel_size)
        ]
        
        # 打印调试信息
        print(f"Label: {cls}")
        print(f"Pixel Center: ({x_pix_center}, {y_pix_center})")
        print(f"Geo Coordinates: {geo_coords}")
        
        # 创建多边形
        poly = Polygon(geo_coords)
        wkt = poly.wkt
        feature = ogr.Feature(layer.GetLayerDefn())
        feature.SetField("Class", cls)
        geom = ogr.CreateGeometryFromWkt(wkt)
        feature.SetGeometry(geom)
        layer.CreateFeature(feature)
        feature = None
    
    data_source = None

# 使用示例
tif_path = "tif_path.tif"
tfw_path = "tfw_path.tfw"
label_path = "yolo.txt"
output_shp_path = "output_dir_path.shp"

top_left_x, top_left_y, x_pixel_size, y_pixel_size = read_tfw(tfw_path)
img_width, img_height = read_image_size(tif_path)
labels = read_yolo_labels(label_path)
create_shapefile(output_shp_path, labels, top_left_x, top_left_y, x_pixel_size, y_pixel_size, img_width, img_height)

Label: 0
Pixel Center: (441.946367115126, 6260.792788451136)
Geo Coordinates: [(109.51431555353292, 33.607587767215), (109.51433025913292, 33.607587767215), (109.51433025913292, 33.607571948815), (109.51431555353292, 33.607571948815), (109.51431555353292, 33.607587767215)]
